# Training Data Generation

The input images will be cut randomly into smaller pieces to train the neural network.

Press 'Run' (above) or Shift+Enter.

In [ ]:
from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
import matplotlib.pyplot as plt
import tkinter as tk
import os
import random
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%gui tk

from tifffile import imread
from tkinter import filedialog, simpledialog
from csbdeep.utils import axes_dict, plot_some, plot_history
from csbdeep.utils.tf import limit_gpu_memory
from csbdeep.data import RawData, create_patches
from csbdeep.io import load_training_data
from csbdeep.models import Config, CARE
from IPython.display import clear_output
from csbmod import load_training_data_direct

root = tk.Tk()
root.withdraw()
root.call('wm', 'attributes', '.', '-topmost', True)

# Ask for folder path
directory = filedialog.askdirectory(title="Choose the ForTraining directory from ImageJ")

def get_tile_num():
    # Return the next power of 2 from int input.
    target = 50000
    denom = 50000 / len(os.listdir(directory + "/HighRes/")) / 4
    return 2**(round(denom) - 1).bit_length()

# Look for first tif file
for file in os.scandir(directory + '/HighRes/'):
    if file.name.endswith(".tif"):
        break

# Open tif file
y = imread(directory + '/HighRes/' + file.name)
x = imread(directory + '/LowRes1/' + file.name)

print('Showing ' + file.name)
print('Image size =', x.shape)

chan = x.shape[1]

if chan > 2:
    chanplt = 2
else:
    chanplt = chan
    
plt.figure(figsize=(16,10))
plot_some(np.stack([np.moveaxis(x,1,-1)[5,:,:,0:chanplt],np.moveaxis(y,1,-1)[5,:,:,0:chanplt]]),
          title_list=[['Low Resolution','High Resolution']], 
          pmin=2,pmax=99.8);

modelname = None;

In [ ]:
# If images match, then run this cell.
if modelname == None:
    modelname = list()
    for i in range(chan):
        modelname.append(input("Model name for channel " + str(i+1) + "? "))

print("Creating patches")

len(os.listdir(directory + "/HighRes/"))

raw_data = RawData.from_folder(
    basepath=directory,
    source_dirs=['LowRes1', 'LowRes2', 'LowRes3'],
    target_dir=directory + '/HighRes',
    axes='ZCYX')

X, Y, XY_axes = create_patches (
    raw_data            = raw_data,
    patch_size          = (8,chan,96,96),
    n_patches_per_image = get_tile_num(),
)

# Two random flips
from tqdm import tqdm
for i in range(1):
    print("Augmenting: flip round " + str(i+1))
    for j in tqdm(range(X.shape[0])):
        n = random.randint(0,3)
        if n == 0: # Do nothing
            pass
        else:
            X[j] = np.flip(X[j], n)
            Y[j] = np.flip(Y[j], n)

# Plus some rotations
print("Augmenting: rotate")
for i in tqdm(range(X.shape[0])):
    n = random.randint(0,3)
    if n == 0: # Do nothing
        pass
    else:
        X[i] = np.rot90(X[i], n, (2,3))
        Y[i] = np.rot90(Y[i], n, (2,3))

for i in range(2):
    plt.figure(figsize=(16,4))
    sl = slice(8*i, 8*(i+1)), 0
    plot_some(X[sl],Y[sl],title_list=[np.arange(sl[0].start,sl[0].stop)])
    plt.show()
None;

# Train each channel separately.
for i in range(chan):
    print("Training channel " + str(i+1))
    (Xchan,Ychan), (X_val,Y_val), axes = load_training_data_direct(
        X[:,i,:,:].reshape((X.shape[0],1,X.shape[2],X.shape[3],X.shape[4])),
        Y[:,i,:,:].reshape((Y.shape[0],1,Y.shape[2],Y.shape[3],Y.shape[4])),
        validation_split=0.01, verbose=True, axes='SCZYX')
    
    c = axes_dict(axes)['C']
    config = Config(axes, n_channel_in=Xchan.shape[c], n_channel_out=Ychan.shape[c],
                    train_steps_per_epoch=150, 
                    unet_n_depth=3, train_batch_size=64,
                    train_learning_rate=0.0008, train_epochs=20,
                    train_reduce_lr={'factor':0.5, 'patience': 4, 'min_delta':0})
    # print(config)
    
    vars(config)
    model = CARE(config, modelname[i], basedir='models')

    history = model.train(Xchan,Ychan, validation_data=(X_val,Y_val))
    clear_output()

## The progress bar is not indicative of the entire training process. It will take 1-2 hours.